In [24]:
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes,
    Graph,
    get_uri_name,
)
from pyshacl import validate
from semantic_mpc_interface.utils import query_to_df
from semantic_mpc_interface.namespaces import BRICK, RDF, S223, A, HPFS, REF, SH
from rdflib import URIRef, Literal
from buildingmotif import BuildingMOTIF, get_building_motif
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone
import json
# SELECT ONTOLOGY 
ontology = 'brick'
# base path
base_path = f'{ontology}'

In [25]:
# Please disregard excessive outputs (logging and warnings) from bmotif
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load the semantic model of our building 
g = Graph()
g.parse('post-processed-model-normal.ttl', format = 'ttl')

# Load the SHACL from our library 
# This lets us infer what structures in the semantic model match the shapes for our controller
handler = SHACLHandler(ontology=ontology, template_dir = 'templates')
handler.generate_shapes()
handler.save_shapes(f'{base_path}/shapes.ttl')
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")

inferred_graph = handler.infer(g)


# Save inferred graph
inferred_graph.serialize(f"inferred_graph.ttl")

<Graph identifier=Nb3fe9b142ce1411792cad27914f84331 (<class 'rdflib.graph.Graph'>)>

In [27]:
# Get our semantic export from the CDL 
data = """
@prefix bldg: <https://BESTESTAir.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix hpfs: <urn:hpflex/shapes#> .
@prefix s223: <http://data.ashrae.org/standard223#> .

@prefix ctrl: <urn:mycontroller.urn#> .

ctrl:zone_ratcheting hpfs:controls hpfs:zone ;
    hpfs:uses ctrl:TZonHeaSetCur, ctrl:TZonCooSetCur, ctrl:occSta,
        ctrl:TZonHeaSetCom, ctrl:TZonCooSetCom, ctrl:TZon .
    
ctrl:occSta a s223:FunctionInput;
    hpfs:binds hpfs:occ .
        
ctrl:TZonHeaSetCur a s223:FunctionInput;
    hpfs:binds hpfs:heaset .

ctrl:TZonCooSetCur a s223:FunctionInput;
    hpfs:binds hpfs:cooset .

ctrl:TZon a s223:FunctionInput;
    hpfs:binds hpfs:temp .

ctrl:TZonCooSetCom a s223:FunctionOutput;
    hpfs:binds hpfs:cooset .

ctrl:TZonHeaSetCom a s223:FunctionOutput;
    hpfs:binds hpfs:heaset .
"""

In [40]:
# Use the semantic export from CDL to get the bindings to our semantic model 
# Uses the hpfs:controls predicate to determine what template the controller is targetting 
# Uses binds to say which points of that target template need to be bound to the controller
ctrl_graph = Graph()
ctrl_graph.parse(data=data, format='turtle')
target_template_uri = list(ctrl_graph[:HPFS.controls:])[0][1]
target_template_name = get_uri_name(ctrl_graph, target_template_uri)
print('looking for template: ',target_template_name)

# Load the graph and get all the opjects matching this template
loader = LoadModel(f"inferred_graph.ttl", 
                   template_dict={'control_target': target_template_name}, 
                   ontology = ontology,
                   template_dir = 'templates')

# runs a query for our controller template, trying to get all the data the template/shape calls for
site_data = loader.get_all_building_objects()

# lets check what bindings we have (spoiler: it will be empty because we don't have occupancy in our model)
print(site_data)

looking for template:  zone2
{'control_target': [zone2(name='https://BESTESTAir.urn#flo_Wes', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonWes_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=[Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')], unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=[Reference(ref_type='NormalReference', name='TZonCooSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonCooSet_u')], unit='http://qudt.org/schema/qudt/K'), ), zone2(name='https://BESTESTAir.urn#flo_Sou', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonSou_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=[Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPT

In [29]:
# We know we have zones in our model, but we aren't getting their data, we should run SHACL to see WHY
# Target the shacl to our zones and see whats wrong 
for zone in inferred_graph.subjects(A, BRICK.Zone):
    inferred_graph.add((zone, A, HPFS.zone))
result, validation_graph, report  = validate(inferred_graph + handler.shapes_graph, shapes_graph=handler.shapes_graph)

# We can see that the zone occupancy is missing, which also means our zones have too few points 
print(report)

Validation Report
Conforms: False
Results (8):
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: hpfs:zone_occ
	Focus Node: bldg:flo_Nor
	Result Path: brick:hasPoint
	Message: Focus node does not conform to shape MinCount 1: hpfs:zone_occ-1
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: hpfs:zone_occ
	Focus Node: bldg:flo_Eas
	Result Path: brick:hasPoint
	Message: Focus node does not conform to shape MinCount 1: hpfs:zone_occ-1
Constraint Violation in QualifiedValueShapeConstraintComponent (http://www.w3.org/ns/shacl#QualifiedMinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: hpfs:zone_occ
	Focus Node: bldg:flo_Wes
	Result Path: brick:hasPoint
	Message: Focus node does not conform to shape MinCount 1: hpfs:zone_occ-1
Constraint Vio

In [30]:
# Now, we need to edit the application in normal so that we can work without occupancy data
# We also need to edit the semantic representation we need. 
# See below, we are referencing hpfs:zone2 (which does not have an occupancy point) and we no longer ask for the occ point 
# We also reference an occupancy controller, so that we can link the occupancy point to the controller.
data = """
@prefix bldg: <https://BESTESTAir.urn#> .
@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix hpfs: <urn:hpflex/shapes#> .
@prefix s223: <http://data.ashrae.org/standard223#> .

@prefix ctrl: <urn:mycontroller.urn#> .

ctrl:zone_ratcheting hpfs:controls hpfs:zone2 ;
    hpfs:uses ctrl:TZonHeaSetCur, ctrl:TZonCooSetCur,
        ctrl:TZonHeaSetCom, ctrl:TZonCooSetCom, ctrl:TZon .
    
ctrl:TZonHeaSetCur a s223:FunctionInput;
    hpfs:binds hpfs:heaset .

ctrl:TZonCooSetCur a s223:FunctionInput;
    hpfs:binds hpfs:cooset .

ctrl:TZon a s223:FunctionInput;
    hpfs:binds hpfs:temp .

ctrl:TZonCooSetCom a s223:FunctionOutput;
    hpfs:binds hpfs:cooset .

ctrl:TZonHeaSetCom a s223:FunctionOutput;
    hpfs:binds hpfs:heaset .

ctrl:occupancy hpfs:controls hpfs:zone ;
    hpfs:uses ctrl:TCooSet, ctrl:THeaSet .

ctrl:TCooSet a s223:FunctionInput;
    hpfs:binds hpfs:cooset .

ctrl:THeaSet a s223:FunctionInput;
    hpfs:binds hpfs:heaset .
"""

In [31]:
# Get our graph and the name of the template representing what we intend to control
ctrl_graph = Graph()
ctrl_graph.parse(data=data, format='turtle')
target_template_uri = list(ctrl_graph[:HPFS.controls:])[0][1]
target_template_name = get_uri_name(ctrl_graph, target_template_uri)
print('looking for template: ',target_template_name)

# Load the graph and get all the opjects matching this template, as before
loader = LoadModel(f"inferred_graph.ttl", 
                   template_dict={'control_target': target_template_name}, 
                   ontology = ontology,
                   template_dir = 'templates')

# runs a query for our controller template, which is the zone template. 
site_data = loader.get_all_building_objects()

# we have our new bindings!
print(site_data)

looking for template:  zone2
{'control_target': [zone2(name='https://BESTESTAir.urn#flo_Wes', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonWes_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=[Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonHeaSet_u')], unit='http://qudt.org/schema/qudt/K'), cooset=Value(value=[Reference(ref_type='NormalReference', name='TZonCooSet'), Reference(ref_type='BOPTestReference', name='hvac_oveZonSupWes_TZonCooSet_u')], unit='http://qudt.org/schema/qudt/K'), ), zone2(name='https://BESTESTAir.urn#flo_Sou', temp=Value(value=[Reference(ref_type='NormalReference', name='TZon'), Reference(ref_type='BOPTestReference', name='hvac_reaZonSou_TZon_y ')], unit='http://qudt.org/schema/qudt/K'), heaset=Value(value=[Reference(ref_type='NormalReference', name='TZonHeaSet'), Reference(ref_type='BOPT

In [32]:
# Look at controller model to get our controlers, their inputs, and their outputs. 
control_targets = site_data['control_target']
controller_dict = {}
for controller,_ in ctrl_graph[:HPFS.controls:]:
    controller_name = get_uri_name(ctrl_graph, controller)
    controller_dict[controller_name] = {}
    controller_dict[controller_name]['inputs'] = inputs = {}
    controller_dict[controller_name]['outputs'] = outputs = {}
    for point in ctrl_graph[controller:HPFS.uses:]:
        if (point, A, S223.FunctionInput) in ctrl_graph:
            input_name = get_uri_name(ctrl_graph, point)
            inputs[input_name] = get_uri_name(ctrl_graph, ctrl_graph.value(point, HPFS.binds))
        elif (point, A, S223.FunctionOutput) in ctrl_graph:
            output_name = get_uri_name(ctrl_graph, point)
            outputs[output_name] = get_uri_name(ctrl_graph, ctrl_graph.value(point, HPFS.binds))

In [33]:
controller_dict

{'zone_ratcheting': {'inputs': {'TZonHeaSetCur': 'heaset',
   'TZonCooSetCur': 'cooset',
   'TZon': 'temp'},
  'outputs': {'TZonHeaSetCom': 'heaset', 'TZonCooSetCom': 'cooset'}},
 'occupancy': {'inputs': {'TCooSet': 'cooset', 'THeaSet': 'heaset'},
  'outputs': {}}}

In [34]:
# Get the bindings from the semantic model for each input and output per controller
def get_normal_reference(ctrl_point_dict, control_targets = control_targets):
    input_bindings = {}
    for target in control_targets:
        for io, type in ctrl_point_dict.items():
            if hasattr(target, type):
                value_obj = getattr(target, type)
                for ref in value_obj.value:
                    if hasattr(ref, 'ref_type') and ref.ref_type == 'NormalReference':
                        input_bindings[io] = ref.name
    return input_bindings

In [35]:
get_normal_reference(controller_dict['zone_ratcheting']['outputs'])

{'TZonHeaSetCom': rdflib.term.Literal('TZonHeaSet', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')),
 'TZonCooSetCom': rdflib.term.Literal('TZonCooSet', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))}

In [36]:
# Filling in the app.json template for normal with the semantic information from CDL and our model 

def query_formatter(equip_refs, input_refs):
    return {
        "type": "POINT_QUERY",

        "query": {
            "and": [
                {
                    "or": [
                        {
                            "field": {
                                "property": "equipRef",
                                "text": str(ref)
                            }
                        } for ref in sorted(equip_refs)
                    ]
                },
                {
                    "or": [
                        {
                            "field": {
                                "property": "class",
                                "text": str(ref_class)
                            }
                        } for ref_class in input_refs.values()
                    ]
                }
            ]
        },
        "label_attribute": "class",
        "groups": {
            "keys": ["equipRef"]
        },
        "query_mode": "POINT_QUERY_MODE_ADVANCED"
    }

# Fill in the queries using our IO 
def create_normal_queries(control_targets, inputs, outputs):
    # TODO: Add zone references as another normal reference to the model, right now just extracting from uri
    equip_refs = set()
    for target in control_targets:
        target_name = target.name.split('#')[-1]
        if '_' in target_name:
            equip_ref = target_name.split('_')[-1]
            equip_refs.add(equip_ref)
    
    input_refs = get_normal_reference(inputs)
    output_refs = get_normal_reference(outputs)
    
    query = query_formatter(equip_refs, input_refs)
    command = query_formatter(equip_refs, output_refs)
    
    return query, command 

# Create the flow edges from our io 
def create_flow_edges(controller_dict, query_id, command_id):
    flow_edges = []
    for controller in controller_dict.keys():
        inputs = controller_dict[controller]['inputs']
        input_refs = get_normal_reference(inputs)
        outputs = controller_dict[controller]['outputs']
        output_refs = get_normal_reference(outputs)
        id = controller_dict[controller]['id']
        for ctrl_name, ref_name in input_refs.items():
            flow_edges.append({
                "from": query_id,
                "to": id,
                "fromName": f"flow-source-handle-{ref_name}",
                "toName": f"flow-target-handle-{ctrl_name}"
            })
        for io in output_refs:
            flow_edges.append({
                "from": id,
                "to": command_id,
                "fromName": f"flow-source-handle-{ctrl_name}",
                "toName": f"flow-target-handle-{ref_name}"
            })
    return flow_edges

In [37]:
app_template = json.loads(open('app-template.json').read())

In [38]:
# Filling in the normal controller template 
query, command = create_normal_queries(control_targets, inputs, outputs)
flow_nodes = app_template['flowNodes']
for node in flow_nodes:
    if node['name'] == 'query':
        query_id = node['id']
        node['query'] = query
    elif node['name'] == 'command':
        command_id = node['id']
        node['command'] = command
    if node['name'] in controller_dict.keys():
        controller_dict[node['name']]['id'] = node['id']
        
new_flow_edges = create_flow_edges(controller_dict, query_id, command_id)

app_template['flowEdges'] = app_template['flowEdges'] + new_flow_edges


with open('app-completed.json', 'w') as f:
    json.dump(app_template, f, indent=2)

In [39]:
# Can look at generated. query, command, flow edges
from pprint import pprint
pprint(query)
pprint(command)
pprint(new_flow_edges)

{'groups': {'keys': ['equipRef']},
 'label_attribute': 'class',
 'query': {'and': [{'or': [{'field': {'property': 'equipRef', 'text': 'Eas'}},
                           {'field': {'property': 'equipRef', 'text': 'Nor'}},
                           {'field': {'property': 'equipRef', 'text': 'Sou'}},
                           {'field': {'property': 'equipRef', 'text': 'Wes'}}]},
                   {'or': [{'field': {'property': 'class',
                                      'text': 'TZonCooSet'}},
                           {'field': {'property': 'class',
                                      'text': 'TZonHeaSet'}}]}]},
 'query_mode': 'POINT_QUERY_MODE_ADVANCED',
 'type': 'POINT_QUERY'}
{'groups': {'keys': ['equipRef']},
 'label_attribute': 'class',
 'query': {'and': [{'or': [{'field': {'property': 'equipRef', 'text': 'Eas'}},
                           {'field': {'property': 'equipRef', 'text': 'Nor'}},
                           {'field': {'property': 'equipRef', 'text': 'Sou'}},
   